# YOUR PROJECT TITLE

> **Note the following:** 
> 1. This is *not* meant to be an example of an actual **data analysis project**, just an example of how to structure such a project.
> 1. Remember the general advice on structuring and commenting your code from [lecture 5](https://numeconcopenhagen.netlify.com/lectures/Workflow_and_debugging).
> 1. Remember this [guide](https://www.markdownguide.org/basic-syntax/) on markdown and (a bit of) latex.
> 1. Turn on automatic numbering by clicking on the small icon on top of the table of contents in the left sidebar.
> 1. The `dataproject.py` file includes a function which will be used multiple times in this notebook.

Imports and set magics:

In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
import numpy as np

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import dataproject

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Read and clean data

Import the excelfile of the deescription and the data

In [16]:
meddef = pd.read_excel('medpriser.xlsx')
meddef

,Unnamed: 0,Unnamed: 1
0,På baggrund af data fra Lægemiddelstyrelsen op...,NaN
1,NaN,NaN
2,Ord og begreber i opgørelsen:,NaN
3,ATC,Lægemiddelklassifikation som fastsættes af WHO...
4,Lægemiddel,Lægemidlets handelsnavn. Navnet fra den senest...
5,Varenummer,Lægemidlets varenummer som også er trykt på pa...
6,Pakning,"Pakningsstørrelse, f.eks. 100 stk."
7,Styrke,"Lægemidlets styrke, f.eks. 5 mg eller 0,01%. F..."
8,Form,"Lægemiddelform, f.eks. tabletter, kapsler, mik..."
9,Firma,Distributør af lægemidlet.


In [20]:
medprice = pd.read_excel('meddata.xlsx')

SyntaxError: invalid syntax (<ipython-input-20-f68b0d8a8536>, line 2)

Clean data: Are there any missing variables

In [22]:
medprice.head()

,ATC,Lægemiddel,Varenummer,Pakning,Styrke,Form,Firma,Indikator,20150202,20150216,...,20191118,20191202,20191216,20191230,20200113,20200127,20200210,20200224,20200309,20200323
0,A01AA01,Bifluorid,42846,4 g + solvens,NaN,dentalsuspension,Voco,AIP,407.36,407.36,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A01AA01,Bifluorid,42846,4 g + solvens,NaN,dentalsuspension,Voco,AUP,570.25,570.25,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A01AA01,Bifluorid,42846,4 g + solvens,NaN,dentalsuspension,Voco,DDD,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A01AA01,Bifluorid,42846,4 g + solvens,NaN,dentalsuspension,Voco,AUP_pr_DDD,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A01AA01,Bifluorid,43158,10 g,NaN,dentalsuspension,Voco,AIP,602.07,602.07,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
medprice.isnull()

,ATC,Lægemiddel,Varenummer,Pakning,Styrke,Form,Firma,Indikator,20150202,20150216,...,20191118,20191202,20191216,20191230,20200113,20200127,20200210,20200224,20200309,20200323
0,False,False,False,False,True,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,True
1,False,False,False,False,True,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,True
2,False,False,False,False,True,False,False,False,True,True,...,True,True,True,True,True,True,True,True,True,True
3,False,False,False,False,True,False,False,False,True,True,...,True,True,True,True,True,True,True,True,True,True
4,False,False,False,False,True,False,False,False,False,False,...,True,True,True,True,True,True,True,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64863,False,False,False,False,False,False,False,False,True,True,...,True,True,True,True,True,True,True,True,True,True
64864,False,False,False,False,False,False,False,False,True,True,...,True,True,True,True,True,True,True,True,True,True
64865,False,False,False,False,False,False,False,False,True,True,...,True,True,True,True,True,True,True,True,True,True
64866,False,False,False,False,False,False,False,False,True,True,...,True,True,True,True,True,True,True,True,True,True


The isnull method correctly recognizes the missing values of the medicine prices.

Coverting the data to long format